In [102]:
from pathlib import Path
path = Path(r"D:\Programme\UBUNTU\Ubuntu.1604.2017.711.0_v1\rootfs\root\com.android.calendar\events-22-55-08\data.csv")

In [103]:
csvdata = []
import csv
with open(path,"r") as fp:
    reader = csv.DictReader(fp,skipinitialspace=True)
    #data = next(reader)
    for row in reader: # Iterate the remaining rows
        csvdata.append(row)
#csvdata

In [104]:
#creates dictonary that contains the events seperated by calendar 
csvcalendars = {} 
# = {calendar1Name: [Event1,...],calendar2Name: [Event1,...]}

for i in range(len(csvdata)):
    key = csvdata[i]['calendar_displayName']  # key = calendarname
    value = csvdata[i]#value = event
    
    if key not in csvcalendars:
        csvcalendars[csvdata[i]['calendar_displayName']] = [value]
    elif type(csvcalendars[key])==list:
         csvcalendars[key].append(value)
    else:
        csvcalendars[key] = [csvcalendars[key][0],value]
csvcalendars

{'S4-R4': [OrderedDict([('sync_data10', 'NULL'),
               ('cal_sync6', 'NULL'),
               ('rrule', 'FREQ=YEARLY;WKST=MO'),
               ('sync_data6', 'NULL'),
               ('cal_sync8', 'NULL'),
               ('eventTimezone', 'Europe/Berlin'),
               ('hasAttendeeData', '1'),
               ('_sync_id', 'NULL'),
               ('customAppPackage', 'NULL'),
               ('originalInstanceTime', 'NULL'),
               ('sync_data2', 'NULL'),
               ('allowedReminders', '0 1'),
               ('uid2445', 'NULL'),
               ('calendar_timezone', 'NULL'),
               ('dirty', '1'),
               ('originalAllDay', 'NULL'),
               ('cal_sync10', 'NULL'),
               ('exrule', 'NULL'),
               ('calendar_color', '-51390'),
               ('cal_sync3', 'NULL'),
               ('lastDate', 'NULL'),
               ('canOrganizerRespond', '1'),
               ('guestsCanSeeGuests', '1'),
               ('rdate', 'NULL'),
        

In [105]:
import icalendar
from icalendar import Calendar, Event, vText,  vDuration
from datetime import datetime
import random

def display(cal):
    return cal.to_ical().decode('utf-8').replace('\r\n', '\n').strip()

In [106]:
allDay = 0 
def setAllday(b):
    global allDay
    allDay = b
def getAllday(): return bool(allDay)

In [107]:
import time, pytz #used for time only berlin for now | improve with localize



def timestamp_to_timeValue(epochTime):
    if(allDay):
        return timestamp_to_date(epochTime)
    elif(allDay==False):
        return timestamp_to_time(epochTime)
    else:
        raise ValueError

def timestamp_to_date(epochTime):
    return  datetime.fromtimestamp(float(epochTime)/1000.).date()#.astimezone(pytz.timezone('UTC')).replace(tzinfo=None)#.replace(tzinfo=pytz.timezone('Europe/Berlin'))#'Europe/Berlin'))
#print(timestamp_to_date('1559174400000'))

def timestamp_to_time(epochTime):
    return  datetime.fromtimestamp(float(epochTime)/1000.).astimezone(pytz.timezone('UTC')).replace(tzinfo=None)#.replace(tzinfo=pytz.timezone('Europe/Berlin'))#'Europe/Berlin'))
#timestamp_to_time('1559174400000')+


In [108]:
timestamp_to_timeValue('1559174400000')

datetime.datetime(2019, 5, 30, 0, 0)

In [109]:
header2componentMap = {
        "title"        :lambda v:("summary",v),
        "description"  :lambda v:("description",v),
        "eventLocation":lambda v:("location",v),
        "dtstart"      :lambda t:("dtstart",timestamp_to_timeValue(t)),
        "dtend"        :lambda t:("dtend",timestamp_to_timeValue(t)),
        #"duration"     :lambda d:("duration",vDuration.from_ical(d)), #doesnt work because it requires a pt infront of time values but there is only a p
        #allday is set when no end or duration is given doesnt work yet sets day from 2am to 2am
        #"rrule"        :lambda rule:("rrule",rule),
    }

def mapHeader2Component(headerKey,value):
    return header2componentMap[headerKey](value)
#print(*mapHeader2Component("title","1"))


In [116]:
icscalendars = {}
for calendarName in csvcalendars:
    icscalendars[calendarName] = Calendar()
    icscalendars[calendarName].add('prodid', '-//My calendar//')
    icscalendars[calendarName].add('version', '2.0')
    icscalendars[calendarName].add('X-WR-TIMEZONE','Europe/Berlin')

    
def add_events_to_ics(calendarName):
    for csvEvent in csvcalendars[calendarName]:
        print("EventName:",csvEvent['title'])
        event = Event()
        event['uid'] = str(random.random())+"@R4"       
        setAllday(csvEvent['allDay'])
        
        for headerKey in csvEvent:   
            
            if headerKey in header2componentMap and csvEvent[headerKey]!='NULL':
                print("   "+headerKey+" exists")
                print("       set to:"+str(mapHeader2Component(headerKey,csvEvent[headerKey])))
                event.add(*mapHeader2Component(headerKey,csvEvent[headerKey]))
                
        icscalendars[calendarName].add_component(event)
        
add_events_to_ics("S4-R4")

EventName: Subject
   dtstart exists
       set to:('dtstart', datetime.date(2019, 1, 8))
   description exists
       set to:('description', 'Description')
   title exists
       set to:('summary', 'Subject')
   eventLocation exists
       set to:('location', 'Location')
EventName: Mein 18.
   dtstart exists
       set to:('dtstart', datetime.date(2019, 5, 29))
   dtend exists
       set to:('dtend', datetime.date(2019, 5, 30))
   title exists
       set to:('summary', 'Mein 18.')
   eventLocation exists
       set to:('location', '@Home')
EventName: AHAHAHAHAHAHAHASUBJECT
   dtstart exists
       set to:('dtstart', datetime.date(2019, 1, 9))
   dtend exists
       set to:('dtend', datetime.date(2019, 1, 9))
   description exists
       set to:('description', 'KEKEKEKKEKEKKEDESCRIPTION')
   title exists
       set to:('summary', 'AHAHAHAHAHAHAHASUBJECT')
   eventLocation exists
       set to:('location', 'AhahahhaLOCATUON')


In [111]:
print(display(icscalendars["S4-R4"]))

BEGIN:VCALENDAR
VERSION:2.0
PRODID:-//My calendar//
X-WR-TIMEZONE:Europe/Berlin
BEGIN:VEVENT
SUMMARY:Subject
DTSTART;VALUE=DATE:20190108
UID:0.6615303405666955@R4
DESCRIPTION:Description
LOCATION:Location
END:VEVENT
BEGIN:VEVENT
SUMMARY:Mein 18.
DTSTART;VALUE=DATE:20190529
DTEND;VALUE=DATE:20190530
UID:0.4185895105430397@R4
LOCATION:@Home
END:VEVENT
BEGIN:VEVENT
SUMMARY:AHAHAHAHAHAHAHASUBJECT
DTSTART;VALUE=DATE:20190109
DTEND;VALUE=DATE:20190109
UID:0.8680008051083791@R4
DESCRIPTION:KEKEKEKKEKEKKEDESCRIPTION
LOCATION:AhahahhaLOCATUON
END:VEVENT
END:VCALENDAR


In [112]:
import tempfile, os
directory = Path(r"")
f = open(os.path.join(directory, 'MYFILEYEEEEEEEEEE.ics'), 'wb') #(filename,mode) wb indicates that the file is opened in binary mode since its a binary file
f.write(icscalendars["S4-R4"].to_ical())
f.close()